In [2]:
%%capture
from sys import path
path.insert(0, '../static')
from preventive import Preventive, PreventiveCommune
from re import sub
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from openpyxl import Workbook
from openpyxl.styles.colors import Color
from openpyxl.styles.fills import PatternFill
from openpyxl.styles import Font
from openpyxl.styles.alignment import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles.borders import Border, Side


# Build Excel Datim / Communes

In [4]:
%%capture
datim = Preventive()

In [5]:
my_blue = Color(rgb="0000FF")
blue_fill = PatternFill(patternType="solid",fgColor=my_blue)
my_red = Color(rgb="FF0000")
red_fill = PatternFill(patternType="solid",fgColor=my_red)
bold_12 = Font(size=12,bold=True)
text_datim_title = Alignment(horizontal="center",vertical="bottom",wrapText=True)
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

In [6]:
%%capture
wb = Workbook()
ws = wb.active
ws.title = datim.who_am_i

ws['A1'].fill = blue_fill


ws.merge_cells("B1:C1")
ws.cell(row=1,column=2).value = Preventive.title()
ws.cell(row=1,column=2).font = bold_12
ws.cell(row=1,column=2).alignment =  text_datim_title


ws['G5'] = datim.total_preventive_violence


rdf_datim_I = dataframe_to_rows(datim.datim_preventive_violence(),index=False,header=True)
for ridx, row in enumerate(rdf_datim_I,1):
    for cidx, value in enumerate(row,1):
        cells = ws.cell(ridx+1,cidx,value)
        cells.border = thin_border
        cells



In [7]:
#wb.sheetnames

def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s



In [9]:
%%capture
worksheets = []
datimcommunes = []
for columns in datim.data_mastersheet.commune.unique():
    globals()[f"ws_{name_handler(columns)}"] = wb.create_sheet(title=PreventiveCommune(f"{columns}").who_am_i)
    globals()[f"datim_{name_handler(columns)}"] =PreventiveCommune(f"{columns}")
    worksheets.append(globals().get(f"ws_{name_handler(columns)}"))
    datimcommunes.append(globals().get(f"datim_{name_handler(columns)}"))


In [10]:
%%capture
worksheets
datimcommunes

In [11]:
%%capture
for wscommunes, dcommunes in zip(worksheets,datimcommunes):
    wscommunes['A1'].fill = blue_fill
    
    wscommunes.merge_cells("B1:C1")
    wscommunes.cell(row=1,column=2).value = PreventiveCommune.title()
    wscommunes.cell(row=1,column=2).font = bold_12
    wscommunes.cell(row=1,column=2).alignment =  text_datim_title
    wscommunes['G5'] = dcommunes.total_preventive_violence
    

    for ridx, row in enumerate(dataframe_to_rows(dcommunes.datim_preventive_violence(),index=False,header=True),1):
        for cidx, value in enumerate(row,1):
            cells = wscommunes.cell(ridx+1,cidx,value)
            cells.border = thin_border
            cells

    

In [12]:
from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [13]:
wb.save(f"./PREVENTIVE_{date_du_jour}.xlsx")
